# Pattern Associator

We recreate the network of the [Pattern Associator tutorial from the CECN1 notebook](https://grey.colorado.edu/CompCogNeuro/index.php/CECN1_Pattern_Associator) exploring how the delta rule works and behave. 

In [ ]:
import numpy as np
import dotdot
import leabra
import graphs

In [ ]:
import random
random.seed(0)

In [ ]:
input_layer  = leabra.Layer(4, name='input_layer')
output_spec  = leabra.LayerSpec(g_i=1.5, ff=0.0, fb=0.5, fb_dt=1/1.4, ff0=0.1)
output_layer = leabra.Layer(2, spec=output_spec, name='output_layer')

In [ ]:
conspec = leabra.ConnectionSpec(proj='full', lrule='leabra', lrate=0.04)
conn    = leabra.Connection(input_layer, output_layer, spec=conspec)

In [ ]:
conn.weights

In [ ]:
network = leabra.Network(layers=[input_layer, output_layer], connections=[conn])

In [ ]:
def event(k, network):
    """Run a minus phase and a plus phase for a given input/output pair"""
    inputs  = [0.0, 0.0, 0.0, 0.0]
    outputs = [0.0, 0.0]
    inputs[k] = 1.0
    outputs[int(k/2)] = 1.0  # desired output

    network.set_inputs({'input_layer': inputs})
    network.set_outputs({'output_layer': outputs})

    # minus phase
    for _ in range(3):
        network.quarter()
        print('g_e', output_layer.g_e)
        print('v_m', output_layer.v_m)
        print('v_m_eq', [u.v_m_eq for u in output_layer.units])
        print('force', output_layer.forced_unit)
        print('act', output_layer.activities)
#        print(input_layer.activities)
        print([u.avg_s_eff for u in input_layer.units])
        
    error = sum((np.array(output_layer.activities) - outputs)**2) 

    # plus phase: the output is set directly
    network.quarter()
    
    return error

In [ ]:
conn.weights
print(output_layer.connections)

In [ ]:
event(0, network)

In [ ]:
conn.weights

In [ ]:
def trial():
    sse = 0.0
    sse += event(0, network)
    sse += event(1, network)
    sse += event(2, network)
    sse += event(3, network)
    return sse / 4

In [ ]:
err = [trial() for _ in range(20)]

In [ ]:
conn.weights

In [ ]:
graphs.line(range(1000), err, "Average error over trials", width=600)